In [4]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")  ###Used for LangSmith tracking
os.environ['LANGCHAIN_TRACING_V2'] ="true"
os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")


c:\Users\mrafi\Documents\LangchainProjects\1-Q&AChatbots\AIProjects\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\mrafi\Documents\LangchainProjects\1-Q&AChatbots\AIProjects\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [6]:
##Tools Creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_wiki)
tools = [wiki, arxiv]
tools


[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\mrafi\\Documents\\LangchainProjects\\1-Q&AChatbots\\AIProjects\\lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))]

In [8]:
##Custom tools [RAG Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter 

In [9]:
loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/get_started/quickstart/#serving-with-langserve")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
vectorstoredb = FAISS.from_documents(documents, embedding=embeddings)
retriever = vectorstoredb.as_retriever()

In [10]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith-search", "Seacrch any info about langsmith")

In [49]:
retriever_tool

Tool(name='langsmith-search', description='Seacrch any info about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000233B84FFD90>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023385FFA5F0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000233B84FFE20>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023385FFA5F0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))

In [11]:
tools = [wiki, arxiv,retriever_tool]

In [38]:
import openai
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama3-8b-8192")

In [23]:
response

'Langsmith is a tool that provides a wrapper around Arxiv.org and has its own search functionality. It can be used to search for information about Langsmith and other topics.'

In [30]:
## Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt


ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024b06afbd95d17449f21316291e0726f13dcfaf990cc0d18087ad689a5'}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplat

In [31]:
##Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [32]:
##Agen executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent= agent, tools=tools,verbose=True)

In [33]:
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [40]:
agent_executor.invoke({"input":"Tell me about lansmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith-search` with `{'query': 'langsmith'}`


Let's ask it what LangSmith is - this is something that wasn't present in the training data so it shouldn't have a very good response.llm.invoke("how can langsmith help with testing?")We can also guide its response with a prompt template.

NotebookInstallationLangSmithBuilding with LangChainLLM ChainDiving DeeperRetrieval ChainDiving DeeperConversation Retrieval ChainAgentDiving DeeperServing with LangServeServerPlaygroundClientNext stepsCommunityDiscordTwitterGitHubPythonJS/TSMoreHomepageBlogYouTubeCopyright ¬© 2024 LangChain, Inc.

= loader.load()text_splitter = RecursiveCharacterTextSplitter()documents = text_splitter.split_documents(docs)embeddings = OpenAIEmbeddings()vector = FAISS.from_documents(documents, embeddings)retriever = vector.as_retriever()# 2. Create Toolsretriever_tool = create_retriever_tool(    retriever,    "langsmith_search",    "Search for information about LangS

{'input': 'Tell me about lansmith',
 'output': 'I\'ve received the output from the tool. Since the tool has provided a lot of information, I will respond accordingly. LangChain is a framework designed to simplify the creation of applications using large language models (LLMs). It appears that LangChain has several components, including LangSmith, LangGraph, and LangServe.\n\nBased on the information provided, it seems that LangChain can be used to create chatbots, extract structured output, and more. It also has a concept of "chains" which seem to be a way of combining different components together.\n\nThe output also mentions LangSmith, which appears to be a tool for searching and retrieving information. It\'s not entirely clear what LangSmith is used for, but it seems to be an important part of the LangChain ecosystem.\n\nI\'ll stop here and not call any more tools, as the output seems to provide a good understanding of LangChain and its components.'}

In [42]:
agent_executor.invoke({"input":"What is machine learning"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks withBased on the tool call result, I'll provide a more detailed response.

Machine learning (ML) is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable systems to learn from data, make decisions, and improve their performance over time. The goal of machine learning is to develop systems that can automatically improve their performance on a task by learning from experience, without being explicitly programmed for each specific instance.

Machine learning is a key technology in many areas, including natural language processing, image and speech recognition, recommender systems, and auton

{'input': 'What is machine learning',
 'output': "Based on the tool call result, I'll provide a more detailed response.\n\nMachine learning (ML) is a subfield of artificial intelligence (AI) that involves the use of algorithms and statistical models to enable systems to learn from data, make decisions, and improve their performance over time. The goal of machine learning is to develop systems that can automatically improve their performance on a task by learning from experience, without being explicitly programmed for each specific instance.\n\nMachine learning is a key technology in many areas, including natural language processing, image and speech recognition, recommender systems, and autonomous vehicles. It has numerous applications in various fields, such as healthcare, finance, marketing, and more.\n\nSome common types of machine learning algorithms include:\n\n1. Supervised learning: The algorithm is trained on labeled data and learns to make predictions based on that data.\n2. 

In [43]:
agent_executor.invoke({"input":"What is this arXiv:1706.03762 research paper all about "})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1706.03762'}`


Published: 2023-08-02
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Lukasz Kaiser, Illia Polosukhin
Summary: The dominant sequence transduction models are based on cThe research paper "Attention Is All You Need" with the ID "1706.03762" is about the development of a new approach to sequence transduction, specifically for machine translation tasks. The authors propose a model that only uses self-attention mechanisms, without relying on traditional recurrent neural networks (RNNs) or convolutional neural networks (CNNs). This approach allows for parallelization and efficiency during training and inference, making it well-suited for large-scale machine translation tasks.

> Finished chain.


{'input': 'What is this arXiv:1706.03762 research paper all about ',
 'output': 'The research paper "Attention Is All You Need" with the ID "1706.03762" is about the development of a new approach to sequence transduction, specifically for machine translation tasks. The authors propose a model that only uses self-attention mechanisms, without relying on traditional recurrent neural networks (RNNs) or convolutional neural networks (CNNs). This approach allows for parallelization and efficiency during training and inference, making it well-suited for large-scale machine translation tasks.'}

In [60]:
import streamlit as st 
from langchain_groq import ChatGroq
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun, DuckDuckGoSearchResults
from langchain.agents import initialize_agent, AgentType
from langchain.callbacks import StreamlitCallbackHandler
import os
from dotenv import load_dotenv

api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

search = DuckDuckGoSearchResults(name="Search")
llm = ChatGroq(groq_api_key=groq_api_key, model_name="llama3-8b-8192", streaming=True)
tools = [search,arxiv,wiki,retriever_tool]
search_agent = initialize_agent(tools,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, handling_parsing_errors=True)
response = search_agent.invoke({"input":"What is this arXiv:1706.03762 research paper all about ","page_content":"None"})

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Parsing LLM output produced both a final answer and a parse-able action:: Thought: It looks like the search query "paper arXiv:1706.03762" returned results that are actually related to the paper "Attention is All You Need" with the arXiv identifier "arXiv:1706.03762". This suggests that the paper exists and is published on arXiv.

Action: Final Answer
Action Input: The paper "Attention is All You Need" with the arXiv identifier "arXiv:1706.03762" is a research paper that introduced the Transformer architecture, which allows machines to understand language and complex information more effectively.

Final Answer: The paper "Attention is All You Need" with the arXiv identifier "arXiv:1706.03762" is a research paper that introduced the Transformer architecture, which allows machines to understand langua

In [57]:
response

{'input': 'what is machine learning?',
 'page_content': 'None',
 'output': 'Here is the final answer:'}